## Introduction to Linear Modeling in Python

- Interpolation: model prediction for points in between the data at hand.
- Extrapolation: model prediction for values outside the range of the data at hand.

#### Object interface

~~~
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

options = dict(marker='o', color='blue')

line = ax.plot(x, y, **options)

_ = ax.set_xlabel('X label')
_ = ax.set_ylabel('Y label')

plt.show()
~~~

### Covariance

- Measures how two variables vary together.

~~~
# deviations of two variables
dx = x - np.mean(x)
dy = y - np.mean(y)

deviation_products = dx * dy

covariance = np.mean(deviation_products)
~~~

### Correlation

~~~
# divide deviations by std. dev.
zx = dx/np.std(x)
zy = dy/np.std(y)

# mean of the normalized deviations
correlation = np.mean(zx*zy)
~~~

### Magnitude vs Direction

- Correlation values: -1 to +1
- Two parts: magnitude (0 to 1) and sign (+ or -)

### Taylor Series

- Things to know:
	1. approximate any curve;
	2. polynomial: $ y = a_0 + a_1 \cdot x + a_2 \cdot x^2 + a_3 \cdot x^3 + ... + a_n \cdot x^n $
	3. often, first order is enough: y = a0 + a1*x

### Model

$y = a_0 + a_1\ x$

- $a_0$: intercept
	- gives the value of y where the line intercepts the  y-axis (x=0).
- $a_1$: slope
	- measure of how sensitive a dependency exists between the two variables ($\displaystyle\frac{\Delta\ x}{\Delta\ y} $).

#### Rescaling vs Dependency

- Conversion between Celsius and Fahrenheit degrees:

~~~
slope = (212 - 32)/(100 - 0) # 180/100 = 9/5
intercept = 32
~~~

#### Residuals

~~~
residuals = y_model - y_data

residuals_squared = np.square(residuals)

rss = np.sum(residuals_squared)
~~~

- RSS: residuals squared sum.
- Optimization: minimization of RSS.

### Least-Squares Optimization

- Setting RSS slope := zero, and some calculus, yields:

$a_1 = \displaystyle\frac{Cov(x,y)}{Var(x)}$

and

$a_0 = \bar{y} - a_1 \cdot \bar{x}$

#### In Numpy

~~~
x_mean = np.sum(x)/len(x)
y_mean = np.sum(y)/len(y)

x_dev = x - x_mean
y_dev = y - y_mean

a1 = np.sum(x_dev * y_dev)/np.sum(x_dev**2)
a0 = y_mean - a1*x_mean
~~~

#### In Scipy

~~~
from scipy import optimize

x_data, y_data = load_data()

def model_func(x, a0, a1):
	return a0 + a1*x

param_opt, param_cov = optimize.curve_fit(model_func, x_data, y_data)

a0 = param_opt[0]
a1 = param_opt[1]
~~~

#### In Statsmodels

~~~
from statsmodels.formula.api import ols

x_data, y_data = load_data()

df = pd.DataFrame(dict(x_name=x_data,y_name=y_data))

model_fit = ols(formula='y_name ~ x_name', data=df).fit()

y_model = model_fit.predict(df)
x_model = x_data

a0 = model_fit.params['Intercept']
a1 = model_fit.params['x_name']
~~~


### Scikit-learn

~~~
from sklearn.linear_model import Linear Regression

# Initialize a general model
model = LinearRegression(fit_intercept=True)

# Load and shape the data
x_raw, y_raw = load_data()
x_data = x_raw.reshape(len(x_raw),1)
y_data = x_raw.reshape(len(y_raw),1)

model_fit = model.fit(x_data,y_data)
~~~

#### Predictions and parameters

~~~
# Extract the linear model parameters

intercept = model.intercept_[0]
slope = model.coef_[0,0]

# Use the model to make predictions
future_x = 2100
future_y = model.predict(future_x)
~~~

### statsmodels

~~~
x, y = load_data()

df = pd.DataFrame(dict(times=x_data,distances=y_data))

model_fit = ols(formula='distances ~ times',data=df).fit()
~~~

### Uncertainty

~~~
a0 = model_fit.params['Intercept'] # intercept
a1 = model_fit.params['times'] # slope

e0 = model_fit.bse['Intercept'] # error in intercept
e1 = model_fit.bse['times'] # error in slope
~~~

### Goodness-of-fit

#### 3 Different Rs

- Building models:
	- **RSS**
- Evaluting models:
	- **RMSE**
	- **R-squared**

#### RMSE

~~~
residuals = y_model - y_data
rss = np.sum(np.square(residuals))

mean_squared_residuals = rss/len(residuals)

# Mean Squared Error
mse = np.mean(np.square(residuals))

# Root Mean Squared Error
rmse = np.sqrt(mse) # Std. Deviation (spread) of residuals
~~~

#### R-squared

- Check how much of the variation on the data is due to the linear trend and how much is not.

~~~
# Deviations
deviations = np.mean(y_data) - y_data
var = np.sum(np.square(deviations))

# Residuals
residuals = y_model - y_data
rss = np.sum(np.square(residuals))

r_squared = 1 - (rss/var)
r = correlation(y_data,y_model)
~~~

#### RMSE vs R-Squared

- RMSE: how much variation is residual
- R-squared: what fraction of variation is linear

When variation due to linear trend is larger than variation due to residuals, the model is better.

### Standard Error

#### Uncertainty in parameters

- Parameter value as center (mean)
- Parameter standard error as spread (std. deviation)
- Std. Error measures parameter uncertainty

#### Computing SE

~~~
df = pd.DataFrame(dict(times=x_data,distances=y_data))

model_fit = ols(formula='distances ~ times',data=df).fit()

a1 = model_fit.params['times'] # slope
a2 = model_fit.params['Intercept'] # intercept

e0 = model_fit.bse['Intercept'] # SE of intercept
e1 = model_fit.bse['times'] # SE of slope
~~~

### Inferential Statistics Concepts

#### Resampling

~~~
# Resampling as Iteration

num_samples = 20
for ns in range(num_samples):
	sample = np.random.choice(population,num_pts)
	distribution_of_means[ns] = sample.mean()

# Sample distribution statistics
mean_of_means = np.mean(distribution_of_means)
stddev_of_means = np.std(distribution_of_means)
~~~

#### Estimation

~~~
# Define Gaussian model function
def gaussian_model(x, mu, sigma):
	coeff_part = 1/(np.sqrt(2 * np.pi * sigma**2))
	exp_part = np.exp(-(x - mu)**2 / (2 * sigma**2))
	return coeff_part * exp_part

# Compute the sample statistics
mean = np.mean(sample)
stdev = np.std(sample)

# Model the population using sample statistics
population_model = gaussian_model(sample,mean,stdev)
~~~

#### Likelihood vs Probability

- Conditional probability: $P(\textrm{outcome }A|\textrm{given }B)$
- Probability: $P(\textrm{data}|\textrm{model})$
	- If a model is given, we ask what is the **probability** that it outputs a particular data point.
- Likelihood: $L(\textrm{model}|\textrm{data})$
	- If the data is given, we ask the what is the **likelihood** that a candidate model could output the particular data set we already have.

- If we had two candidate models, we'd want to choose the one that has the greatest likelihood to output the given data.

#### Computing Likelihood

- Likelihood from parameters

~~~
# Guess parameters
mu_guess = np.mean(sample_distances)
sigma_guess = np.std(sample_distances)

# For each sample point, compute a probability
probabilities = np.zeros(len(sample_distances))
for n, distance in enumerate(sample_distances):
	probabilities[n] = gaussian_model(distance,mu_guess,sigma_guess)

likelihood = np.product(probs)
loglikelihood = np.sum(np.log(probs))
~~~

#### Maximum Likelihood Estimation

~~~
# Create an array of mu guesses
low_guess = sample_mean - 2*sample_stdev
high_guess = sample_mean + 2*sample_stdev
mu_guesses = np.linspace(low_guess,high_guess, 101)

# Compute the loglikelihood for each guess
loglikelihoods = np.zeros(len(mu_guesses)))
for n, mu_guess in enumerate(mu_guesses):
	loglikelihood[n] = compute_loglikelihood(sample_distances,mu_guess,sample_stdev)

# Find the best guess
max_loglikelihood = np.max(loglikelihoods)
best_mu = mu_guesses[loglikelihoods == max_loglikelihood]
~~~

### Bootstrap resampling

~~~
# Use sample as model for population
population_model = august_daily_highs_for_2017

# Simulate repeated data acquisitions by resampling the 'model'
for nr in range(num_resamples):
	bootstrap_sample = np.random.choice(population_model,size=resample_size,replace=True)
	bootstrap_means[nr] = np.mean(bootstrap_sample)
~~~

### Types of Error

1. Measurement error
	- broken sensor, worngly recorded measurement.
2. Sampling bias
	- temperatures only from August, when days are hottest.
3. Random chance

#### Null hypotthesis

- Question: Is our effect due to a relationship or due to random chance?

- Test statistic:

~~~
# Group into early and late times
group_short = sample_distance[times < 5]
group_long = sample_distance[times > 5]

# Resample distributions
resample_short = np.random.choice(group_short,size=500,replace=True)
resample_long = np.random.choice(group_long,size=500,replace=True)

test_statistic = resample_long - resample_short

# Effect size as mean of test statistic distribution
effect_size = np.mean(test_statistic)
~~~

- Shuffle and split:

~~~
# Concatenate and Shuffle
shuffle_bucket = np.concatenate((group_short,group_long))
np.random.shuffle(shuffle_bucket)

# Split in the middle
slice_index = len(shuffle_bucket)//2
shuffled_half1 = shuffle_bucket[:slice_index]
shuffled_half2 = shuffle_bucket[slice_index:]
~~~

- Resample and test again:

~~~
# Resample shuffled populations
shuffled_sample1 = np.random.choice(shuffled_half1,size=500,replace=True)
shuffled_sample2 = np.random.choice(shuffled_half2,size=500,replace=True)

# Recompute effect size
shuffled_test_statistic = shuffled_sample2 - shuffled_sample1
effect_size = np.mean(shuffled_test_statistic)
~~~